In [1]:
import networkx as nx
import gamegraph 
import matplotlib.pyplot as plt
import surewin
import random
import copy

# random.seed(6)  # Sub modular is true optima
random.seed(47)

n = 100
e = 3
k = 2

# Generation of Attack-Defend Game Graph

Game graph: 
- Nodes: 20
- Actions/Node: 3

For each node, 3 outgoing edges are chosen at random. 
Sometimes these may lead to same node, i.e. repeated edges. 
We do not manually remove the duplicate edges because they do not have any consequence on our algorithm. 

Note that the nodes in graph are equally distributed to be P1 and P2 nodes. 
However, we not require all edges from P1 node to lead to a P2 node or vice versa. 
In other words, we do not generate a strictly alternating turn-based game. 


In [2]:
graph = gamegraph.generate_game_graph(n, e)
final = set(random.sample(graph.nodes(), 2))

print(f"Generated Game Graph: <S, Act, T, F> with {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges (of which {len(set(graph.edges()))} are unique), where")
print(f"\n- S = {set(graph.nodes())} with ")
print(f"-- S1 = {set([n for n in graph.nodes() if graph.nodes[n]['turn'] == 1])}")
print(f"-- S2 = {set([n for n in graph.nodes() if graph.nodes[n]['turn'] == 2])}")

print(f"\n- Act is not explicitly captured.")
print(f"\n- T = {set(graph.edges())}")

print(f"\n- F = {set(final)}")

Generated Game Graph: <S, Act, T, F> with 100 nodes and 300 edges (of which 295 are unique), where

- S = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99} with 
-- S1 = {0, 1, 2, 4, 9, 10, 12, 13, 16, 18, 20, 22, 23, 26, 30, 34, 36, 38, 39, 43, 45, 47, 48, 50, 51, 52, 54, 56, 57, 58, 59, 61, 62, 67, 71, 72, 75, 77, 78, 81, 82, 83, 85, 86, 89, 91, 92, 93, 95, 98}
-- S2 = {3, 5, 6, 7, 8, 11, 14, 15, 17, 19, 21, 24, 25, 27, 28, 29, 31, 32, 33, 35, 37, 40, 41, 42, 44, 46, 49, 53, 55, 60, 63, 64, 65, 66, 68, 69, 70, 73, 74, 76, 79, 80, 84, 87, 88, 90, 94, 96, 97, 99}

- Act is not explicitly captured.

- T = {(34, 7), (24, 30), (82, 86), (19, 92), (28, 62

# Sure Winning Region of P2

Compute surewinning region of P2 in reachability game. 

In [3]:
sw = surewin.surewin(graph, final, 2)
print("P2 win: ", len(sw), sw)
print("P1 win: ", graph.number_of_nodes() - len(sw), set(graph.nodes()) - sw)

P2 win:  65 {0, 2, 3, 4, 5, 7, 8, 10, 11, 13, 15, 17, 20, 21, 24, 26, 27, 30, 31, 32, 33, 34, 35, 36, 38, 41, 42, 43, 44, 46, 47, 48, 50, 51, 54, 56, 57, 58, 63, 64, 65, 66, 67, 68, 70, 72, 75, 76, 77, 78, 79, 80, 83, 84, 86, 87, 90, 91, 92, 93, 94, 95, 96, 98, 99}
P1 win:  35 {1, 6, 9, 12, 14, 16, 18, 19, 22, 23, 25, 28, 29, 37, 39, 40, 45, 49, 52, 53, 55, 59, 60, 61, 62, 69, 71, 73, 74, 81, 82, 85, 88, 89, 97}


# Hypergame for Decoy Allocation

The hypergame is generated by first identifying which edges are subjectively rationalizable in P2's mind. And then, we eliminate all actions that are not subj. rationalizable. 


In [4]:
hgame = surewin.surewin_graph(graph, sw, 2)
print(hgame.nodes(data=True))

print(f"Generated Hypergame Graph: <S, Act, T, F> with {hgame.number_of_nodes()} nodes and {hgame.number_of_edges()} edges, where")
print(f"\n- S = {set(hgame.nodes())} with ")
print(f"-- S1 = {set([n for n in hgame.nodes() if hgame.nodes[n]['turn'] == 1])}")
print(f"-- S2 = {set([n for n in hgame.nodes() if hgame.nodes[n]['turn'] == 2])}")

print(f"\n- Act is not explicitly captured.")
print(f"\n- T = {set(hgame.edges())}")

print(f"\n- F = {set(final)}")

[(0, {'turn': 1}), (2, {'turn': 1}), (3, {'turn': 2}), (4, {'turn': 1}), (5, {'turn': 2}), (7, {'turn': 2}), (8, {'turn': 2}), (10, {'turn': 1}), (11, {'turn': 2}), (13, {'turn': 1}), (15, {'turn': 2}), (17, {'turn': 2}), (20, {'turn': 1}), (21, {'turn': 2}), (24, {'turn': 2}), (26, {'turn': 1}), (27, {'turn': 2}), (30, {'turn': 1}), (31, {'turn': 2}), (32, {'turn': 2}), (33, {'turn': 2}), (34, {'turn': 1}), (35, {'turn': 2}), (36, {'turn': 1}), (38, {'turn': 1}), (41, {'turn': 2}), (42, {'turn': 2}), (43, {'turn': 1}), (44, {'turn': 2}), (46, {'turn': 2}), (47, {'turn': 1}), (48, {'turn': 1}), (50, {'turn': 1}), (51, {'turn': 1}), (54, {'turn': 1}), (56, {'turn': 1}), (57, {'turn': 1}), (58, {'turn': 1}), (63, {'turn': 2}), (64, {'turn': 2}), (65, {'turn': 2}), (66, {'turn': 2}), (67, {'turn': 1}), (68, {'turn': 2}), (70, {'turn': 2}), (72, {'turn': 1}), (75, {'turn': 1}), (76, {'turn': 2}), (77, {'turn': 1}), (78, {'turn': 1}), (79, {'turn': 2}), (80, {'turn': 2}), (83, {'turn': 1}),

# Submodularity Formulation

Compute DSWin for each node if it is configured as decoy. 


In [5]:
# Helper class
class Pair(object):
    def __init__(self, n, dswin_n):
        self.n = n
        self.dswin_n = dswin_n
        
    def __repr__(self):
        return f"({self.n}: {self.dswin_n})"
    
    def __len__(self):
        return len(self.dswin_n)


In [6]:
# DSWin Computation
dswin_u = set()
for n in hgame.nodes():
    if n in final:
        continue
        
    dswin_u.add(Pair(n, surewin.surewin(hgame, {n}, 1)))

print("(Node: DSWin)")
for e in dswin_u:
    print(e)
    

(Node: DSWin)
(68: {68})
(30: {24, 30})
(67: {67})
(10: {84, 7, 10, 91, 95})
(13: {0, 65, 66, 3, 4, 8, 42, 13, 80, 83, 86, 57})
(20: {20})
(79: {79})
(46: {46})
(26: {0, 65, 66, 3, 4, 8, 13, 80, 83, 86, 26, 42, 57})
(72: {34, 21, 72, 31})
(44: {44})
(75: {70, 7, 75, 76, 77, 79, 20, 21, 94, 95, 31, 34, 99, 36, 46, 51, 56, 58})
(27: {27})
(43: {80, 43})
(78: {78, 87})
(41: {41})
(77: {70, 7, 76, 77, 79, 20, 21, 94, 95, 31, 34, 99, 36, 46, 51, 56, 58})
(42: {42})
(83: {8, 66, 83})
(56: {56, 20})
(90: {90})
(21: {21})
(15: {15})
(84: {84})
(31: {31})
(80: {80})
(35: {35})
(86: {80, 65, 86})
(36: {56, 36, 20})
(87: {87})
(33: {33})
(38: {0, 65, 66, 3, 4, 8, 13, 80, 83, 84, 86, 26, 91, 38, 42, 57})
(17: {17})
(11: {11})
(4: {0, 65, 66, 3, 4, 8, 13, 80, 83, 86, 42, 57})
(0: {0, 65, 66, 3, 4, 8, 13, 80, 83, 86, 42, 57})
(24: {24})
(7: {7})
(47: {66, 8, 76, 79, 83, 20, 21, 24, 30, 31, 94, 34, 99, 36, 46, 47, 51, 56, 58})
(91: {91, 84})
(92: {0, 65, 66, 3, 4, 8, 13, 80, 83, 84, 86, 26, 91, 92, 3

In [7]:
# Greedy Algorithm (Adds set with most new elements)
# https://homes.cs.washington.edu/~marcotcr/blog/greedy-submodular/
def greedy_cover(universe, subsets, max_k=float("inf")):
    
    # avoid mutation
    tmp_subsets = copy.deepcopy(subsets)
    
    elements = set(e for s in tmp_subsets for e in s.dswin_n)
    largest_subset = max(tmp_subsets, key=lambda x: len(x))
    tmp_subsets.remove(largest_subset)
    
    covered = largest_subset.dswin_n
    uncovered = elements - covered
    cover = [largest_subset.n]
    
    
    # Greedily add the subsets with the most uncovered points
    while len(uncovered) > 0 and len(cover) < max_k:
        subset = max(tmp_subsets, key=lambda s: len(s.dswin_n - covered))
        cover.append(subset.n)
        covered |= subset.dswin_n
        uncovered = elements - covered
        tmp_subsets.remove(subset)
 
    cover.sort()  # for convenience of reading :) 
    return cover, covered

decoys, covered = greedy_cover(hgame.nodes(), dswin_u, k)

print("Decoys:", decoys)
print("Covered Region:", covered, ", length(covered): ", len(covered)) 
print("Uncovered Region:", set(hgame.nodes()) - covered)

Decoys: [47, 92]
Covered Region: {0, 3, 4, 8, 13, 20, 21, 24, 26, 30, 31, 34, 36, 38, 42, 46, 47, 51, 56, 57, 58, 65, 66, 76, 79, 80, 83, 84, 86, 91, 92, 94, 99} , length(covered):  33
Uncovered Region: {2, 5, 7, 10, 11, 15, 17, 27, 32, 33, 35, 41, 43, 44, 48, 50, 54, 63, 64, 67, 68, 70, 72, 75, 77, 78, 87, 90, 93, 95, 96, 98}


# Algorithm 2: Modified GreedyMax



In [8]:
def greedymax(universe, subsets, max_k=float("inf")):
    
    tmp_subsets = copy.deepcopy(subsets)
    elements = set(e for s in tmp_subsets for e in s.dswin_n)
    decoys = set()
    covered = set()
    
    iter_count = 0
    while (len(elements - covered) > 0 and len(decoys) < max_k):
        iter_count += 1
        print(f"Iteration {iter_count}")
        
        nondecoys = elements - decoys
        updated_dswins = list()
        print(f"\tNon-decoy: {nondecoys}")
        
        for n in nondecoys:
            final = list() + list(decoys) + [n]
            pair = Pair(n, surewin.surewin(hgame, set(final), 1))
            updated_dswins.append(pair)                   
            print(f"\t\tPair({n}, surewin.surewin(hgame, {final}, 1)): {pair}")
        
        next_decoy = max(updated_dswins, key=lambda x: len(x))
        decoys.add(next_decoy.n)
        covered.update(next_decoy.dswin_n)
        
        print(f"\tUpdated DSWin_n: {updated_dswins}")
        
        print(f"\tSelected Decoy: {next_decoy.n}")
 
    return decoys, covered

decoys, covered = greedymax(hgame.nodes(), dswin_u, k)

print("Decoys:", decoys)
print("Covered Region:", covered, ", length(covered): ", len(covered)) 
print("Uncovered Region:", set(hgame.nodes()) - covered)

Iteration 1
	Non-decoy: {0, 2, 3, 4, 5, 7, 8, 10, 11, 13, 15, 17, 20, 21, 24, 26, 27, 30, 31, 33, 34, 35, 36, 38, 41, 42, 43, 44, 46, 47, 48, 50, 51, 54, 56, 57, 58, 63, 64, 65, 66, 67, 68, 70, 72, 75, 76, 77, 78, 79, 80, 83, 84, 86, 87, 90, 91, 92, 93, 94, 95, 96, 98, 99}
		Pair(0, surewin.surewin(hgame, [0], 1)): (0: {0, 65, 66, 3, 4, 8, 13, 80, 83, 86, 42, 57})
		Pair(2, surewin.surewin(hgame, [2], 1)): (2: {2, 84})
		Pair(3, surewin.surewin(hgame, [3], 1)): (3: {3})
		Pair(4, surewin.surewin(hgame, [4], 1)): (4: {0, 65, 66, 3, 4, 8, 13, 80, 83, 86, 42, 57})
		Pair(5, surewin.surewin(hgame, [5], 1)): (5: {5})
		Pair(7, surewin.surewin(hgame, [7], 1)): (7: {7})
		Pair(8, surewin.surewin(hgame, [8], 1)): (8: {8})
		Pair(10, surewin.surewin(hgame, [10], 1)): (10: {84, 7, 10, 91, 95})
		Pair(11, surewin.surewin(hgame, [11], 1)): (11: {11})
		Pair(13, surewin.surewin(hgame, [13], 1)): (13: {0, 65, 66, 3, 4, 8, 42, 13, 80, 83, 86, 57})
		Pair(15, surewin.surewin(hgame, [15], 1)): (15: {15

# Comparison with True Optimal

Assuming we can place at max 2 decoys, let us compare our result to winning regions of all acceptable pairs of decoys. 

In [74]:
from itertools import combinations

pairs = combinations(set(hgame.nodes()) - final, k)


dswin = set()
for p in pairs:
    dswin.add(Pair(tuple(p), surewin.surewin(hgame, set(p), 1)))

dswin = list(dswin)
dswin.sort(reverse=True, key=lambda x: len(x.dswin_n))

print("Largest true DSWin:", dswin[0], ", length(dswin): ", len(dswin[0]))


Largest true DSWin: ((75, 92): {0, 3, 4, 7, 8, 13, 20, 21, 26, 31, 34, 36, 38, 42, 46, 51, 56, 57, 58, 65, 66, 70, 75, 76, 77, 79, 80, 83, 84, 86, 91, 92, 94, 95, 99}) , length(dswin):  35


# Plotting 

We want to plot the steps in decision process of Alg.2, if possible.  